# CI

```{admonition} Energy is not complete
:class: tip
This page provide variational calculations of **CI** and **CASSCF**.

The occupation numbers in the outputs serve as reference when good energies are obtained.

Notice that in some bad cases, the energy will be above CCSD(T).

- **SIE4x4** is FullCI.
- **$N_e$ <=  4e** is FCI
- **$N_e$ = 5** is CISDTQ5
- **6 <= $N_e$ <= 9** is CISDTQ
```


In [1]:
import yaml
import numpy as np

set_name = "P30-5"
nof = "CI"
ncwo = "ref"
path = "../{}/{}/{}/".format(ncwo, set_name, nof)
file = open(path + '{}-{}.yaml'.format(set_name, nof, ncwo), 'r')
data = yaml.safe_load(file)

In [2]:
def get_total_MAD_of_whole_set(data):
    ADs = []
    for set in data:
        for system in data[set]:
            ADs.append(data[set][system]["AD"])
    MAD_P30 = np.mean(ADs)
    return MAD_P30

def get_contribution_to_total_MADs(data):
    MADs = []
    for set in data:
        ADs = []
        for system in data[set]:
            ADs.append(data[set][system]["AD"])
        MADs.append(np.sum(ADs)/30)
    return MADs

def get_MADs(data):
    MADs = []
    for set in data:
        ADs = []
        for system in data[set]:
            ADs.append(data[set][system]["AD"])
        MADs.append(np.mean(ADs))
    return MADs

def get_Sets(data):
    Sets = []
    for set in data:
        Sets.append(set)
    return Sets

def get_Descriptions(subsets):
    f_des = open('Sets_description.yaml', 'r')
    d_des = yaml.safe_load(f_des)
    descriptions = []
    for set in subsets:
        descriptions.append(d_des[set])
    return descriptions

In [3]:
from myst_nb import glue
MAD_P30 = get_total_MAD_of_whole_set(data)
glue("MAD_P30", "{:.2f}".format(MAD_P30), display=False)
glue("set_name", set_name, display=False)
glue("nof", nof, display=False)

In [4]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
import ipywidgets as widgets
from IPython.display import display
import py3Dmol

def plot_molecules(data):

    subsets = get_Sets(data)
    nsets = len(subsets)
    dropdown_options = dict(zip(data.keys(), range(nsets)))
    descriptions = get_Descriptions(subsets)

    set_dropdown = widgets.Dropdown(
        options=dropdown_options,
        value=0,
        description='Set:'
    )

    set_box = widgets.VBox()

    set_stack = widgets.Stack()
    set_box.children = [set_dropdown, set_stack]

    widgets.jslink((set_dropdown, 'index'), (set_stack, 'selected_index'))

    boxes = []
    for h,set in enumerate(data):
        set_description = widgets.Text(
            value=descriptions[h],
            description='Description: ',
            style={'description_width': 'max-content'},
            disabled=False
        )

        system_box = widgets.VBox()

        system_label = widgets.Label(value="Reactions")
        system_tabs = widgets.Tab()

        system_box.children = [set_description, system_label, system_tabs]

        system_tabs.children = [widgets.VBox() for system in data[set]]
        for i,system in enumerate(data[set]):
            system_tabs.set_title(i, str(i+1))

            mols_box = widgets.VBox()
            mols_label = widgets.Label(value="Molecules")
            mols_tabs = widgets.Tab()
            mols_box.children = [mols_label, mols_tabs]

            mols_tabs.children = [widgets.VBox() for specie in data[set][system]["Species"]]
            childrens = []
            for j,specie in enumerate(data[set][system]["Species"]):
                url = "url:https://raw.githubusercontent.com/stephengdale/poison-set/refs/heads/main/{}/{}.xyz".format(
                    set_name, specie
                )
                viewer = py3Dmol.view(url, width=280, height=300)
                viewer.setStyle({'stick': {'radius': .1}, 'sphere': {'scale': 0.25}})
                viewer.zoomTo()
                childrens.append(widgets.Output())
                with childrens[-1]:
                    viewer.show()
            mols_tabs.children = childrens
            for j,specie in enumerate(data[set][system]["Species"]):
                mol_name = str(specie)
                mol_name = mol_name.replace(set + "_", "")
                mols_tabs.set_title(j, mol_name)

            species_box = widgets.VBox([widgets.Label("Specie")] + [widgets.Label("{}".format(specie.replace(set + "_", ""))) for specie in data[set][system]["Species"]])
            E_NOF_box = widgets.VBox([widgets.Label("E_CI")] + [widgets.Label("{:>9.3f} Ha".format(data[set][system]["Species"][specie]["E_NOF"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            Stoichiometry_box = widgets.VBox([widgets.Label("Stoichiometry")] + [widgets.Label("{:^15}".format(data[set][system]["Species"][specie]["Count"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            # charge_box = widgets.VBox([widgets.Label("Charge")] + [widgets.Label("{}".format(data[set][system]["Species"][specie]["Charge"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            mult_box = widgets.VBox([widgets.Label("Mult")] + [widgets.Label("{}".format(data[set][system]["Species"][specie]["Multiplicity"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            xyz_box = widgets.VBox([widgets.Label("Geom")] + [widgets.HTML(value="<a href=\"https://raw.githubusercontent.com/stephengdale/poison-set/refs/heads/main/{}/{}.xyz\">XYZ</a>".format(set_name, specie)) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            out_box = widgets.VBox([widgets.Label("Output")] + [widgets.HTML(value="<a href=\"https://raw.githubusercontent.com/felipelewyee/Poison-NOF/refs/heads/main/{}/{}/{}/{}.out\">OUT</a>".format(ncwo, set_name, nof, specie)) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            text_specie_w = widgets.HBox([species_box, E_NOF_box, Stoichiometry_box, mult_box, xyz_box, out_box])

            text_E_NOF_w = widgets.Text(
                value="{:.3f} kcal/mol".format(data[set][system]["dE_NOF"]),
                description='ΔE_CI',
                disabled=False
            )

            text_E_ref_w = widgets.Text(
                value="{:.3f} kcal/mol".format(data[set][system]["dE_Ref"]),
                description='ΔE_Ref',
                disabled=False
            )

            text_AD_w = widgets.Text(
               value="{:.3f} kcal/mol".format(data[set][system]["AD"]),
               description='Abs. Dev.',
               disabled=False
            )

            # text_weight_w = widgets.Text(
            #    value="{:.3f}".format(data[set][system]["Weight"]),
            #    description='Weight',
            #    disabled=False
            # )

            # text_WTMAD2_w = widgets.Text(
            #    value="{:.3f}".format(data[set][system]["WTMAD2"]),
            #    description='WTMAD2',
            #    disabled=False
            # )

            system_tabs.children[i].children = [mols_box, text_specie_w,
                                             text_E_NOF_w, text_E_ref_w,
                                             text_AD_w]#, text_weight_w,
            # text_WTMAD2_w]
        boxes.append(system_box)

    set_stack.children = boxes

    return set_box

In [6]:
nsets = len(list(data.items()))
step = nsets
for i in range(0, nsets, nsets):
    data_n = dict(list(data.items())[i:min(i+step,nsets)])
    tabs = plot_molecules(data_n)
    display(tabs)